In [7]:
!pip install opencv-python-headless pillow

In [8]:
!pip install opencv-python-headless  # Используйте headless версию OpenCV, если стандартная вызывает проблемы в Colab
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import io
import base64
from IPython.display import HTML

In [27]:
import json
import cv2
from PIL import ImageFont, ImageDraw, Image
import numpy as np

def load_text_from_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data['comment']

def split_text(text, words_per_fragment=3):
    words = text[0].split()
    return [' '.join(words[i:i + words_per_fragment]) for i in range(0, len(words), words_per_fragment)]

def add_text_effects(frame, text, position, font_path, font_size):
    img_pil = Image.fromarray(frame)
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype(font_path, font_size)
    shadow_color = 'gray'
    outline_color = 'black'
    text_color = 'white'
    x, y = position
    shadow_offset = 0
    draw.text((x + shadow_offset, y + shadow_offset), text, font=font, fill=shadow_color)
    outline_range = 1
    for dx in range(-outline_range, outline_range + 1):
        for dy in range(-outline_range, outline_range + 1):
            if dx != 0 or dy != 0:
                draw.text((x + dx, y + dy), text, font=font, fill=outline_color)
    draw.text(position, text, font=font, fill=text_color)
    return np.array(img_pil)

def add_text_to_video(video_path, output_path, json_path, font_path, words_per_fragment=5, rotate='clockwise'):
    text = load_text_from_json(json_path)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video stream or file")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    ret, sample_frame = cap.read()
    if rotate == 'clockwise':
        sample_frame = cv2.rotate(sample_frame, cv2.ROTATE_90_CLOCKWISE)
    elif rotate == 'counterclockwise':
        sample_frame = cv2.rotate(sample_frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (sample_frame.shape[1], sample_frame.shape[0]))

    fragments = split_text(text, words_per_fragment)
    total_fragments = len(fragments)
    frames_per_fragment = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) / total_fragments)
    current_fragment_index = 0
    frame_count = 0
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset frame position after sample

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if rotate == 'clockwise':
            frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
        elif rotate == 'counterclockwise':
            frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

        current_fragment = fragments[current_fragment_index]
        frame_with_text = add_text_effects(frame, current_fragment, (50, 50), font_path, 20)
        out.write(frame_with_text)
        frame_count += 1
        if frame_count % frames_per_fragment == 0 and current_fragment_index < total_fragments - 1:
            current_fragment_index += 1

    cap.release()
    out.release()
    print("Video saved successfully")

# Пути и параметры
video_path = '/content/final_combination_3.mp4'
output_path = '/content/output_video.mp4'
json_path = '/content/combined_output (2).json'  # JSON файл с текстом
font_path = '/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf'
add_text_to_video(video_path, output_path, json_path, font_path)

Video saved successfully


In [26]:
import json
import cv2
from PIL import ImageFont, ImageDraw, Image
import numpy as np


def load_text_from_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    comment = data['comment']
    # Обработка случая, когда комментарий является списком
    if isinstance(comment, list):
        comment = ' '.join(comment)  # Объединение списка в одну строку
    return comment

def split_text(text, words_per_fragment=3):
    words = text.split()
    return [' '.join(words[i:i + words_per_fragment]) for i in range(0, len(words), words_per_fragment)]

def add_text_effects(frame, text, position, font_path, font_size):
    img_pil = Image.fromarray(frame)
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype(font_path, font_size)
    shadow_color = 'gray'
    outline_color = 'black'
    text_color = 'white'
    x, y = position
    shadow_offset = 0
    draw.text((x + shadow_offset, y + shadow_offset), text, font=font, fill=shadow_color)
    outline_range = 1
    for dx in range(-outline_range, outline_range + 1):
        for dy in range(-outline_range, outline_range + 1):
            if dx != 0 or dy != 0:
                draw.text((x + dx, y + dy), text, font=font, fill=outline_color)
    draw.text(position, text, font=font, fill=text_color)
    return np.array(img_pil)

def add_text_to_video(video_path, output_path, json_path, font_path, words_per_fragment=5, rotate='clockwise'):
    text = load_text_from_json(json_path)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video stream or file")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    ret, sample_frame = cap.read()
    if not ret:
        print("Error: Cannot read video file")
        return

    if rotate == 'clockwise':
        sample_frame = cv2.rotate(sample_frame, cv2.ROTATE_90_CLOCKWISE)
    elif rotate == 'counterclockwise':
        sample_frame = cv2.rotate(sample_frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (sample_frame.shape[1], sample_frame.shape[0]))

    fragments = split_text(text, words_per_fragment)
    total_fragments = len(fragments)
    frames_per_fragment = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) / total_fragments)
    current_fragment_index = 0
    frame_count = 0
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset frame position after sample

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if rotate == 'clockwise':
            frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        elif rotate == 'counterclockwise':
            frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

        current_fragment = fragments[current_fragment_index]
        frame_with_text = add_text_effects(frame, current_fragment, (50, 50), font_path, 30, frame.shape[1])
        out.write(frame_with_text)
        frame_count += 1
        if frame_count % frames_per_fragment == 0 and current_fragment_index < total_fragments - 1:
            current_fragment_index += 1

    cap.release()
    out.release()
    print("Video saved successfully")
    # Пути и параметры
video_path = '/content/final_combination_3.mp4'
output_path = '/content/output_video.mp4'
json_path = '/content/combined_output (2).json'  # JSON файл с текстом
font_path = '/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf'
add_text_to_video(video_path, output_path, json_path, font_path)


TypeError: add_text_effects() takes 5 positional arguments but 6 were given